In [1]:
import pandas as pd
import os
from selenium import webdriver
import time
import re # 정규식을 활용하여 괄호 안 숫자 추출

#os.chdir("")

In [2]:
data = pd.read_csv("C:/Users/zhqhd/Desktop/BOAZ 공부 자료/ADV/data/Final Res.csv")
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,상일동,상일동역 도전최강달인왕만두
1,이때,한식,한식/백반/한정식,서울특별시 마포구 동교로38안길 7,경의중앙 홍대입구,360.468770,홍대입구,홍대입구역 이때
2,L.A.D,한식,한식/백반/한정식,서울특별시 마포구 잔다리로3안길 23,2호선 홍대입구,583.594176,홍대입구,홍대입구역 L.A.D
3,제이씨에스푸드,한식,한식/백반/한정식,서울특별시 관악구 신림로14길 3,2호선 봉천,860.540255,봉천,봉천역 제이씨에스푸드
4,비지트,한식,한식/백반/한정식,서울특별시 서초구 동광로18길 82,2호선 방배,810.975643,방배,방배역 비지트
...,...,...,...,...,...,...,...,...
135805,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,방화,방화역 얼음과자
135806,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,상일동,상일동역 오이소서울
135807,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,굽은다리,굽은다리역 얼음왕국명일14
135808,얼음과자,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로83길 6,5호선 상일동,1380.011573,상일동,상일동역 얼음과자


In [22]:
items = data["검색어"].head(100)
items

0          상일동역 도전최강달인왕만두
1                홍대입구역 이때
2             홍대입구역 L.A.D
3             봉천역 제이씨에스푸드
4                 방배역 비지트
             ...         
95                역삼역 강남존
96               방화역 버블베어
97    공릉역 서울과학기술대학교제2학생식당
98      어린이대공원역 초밥&참치전문점연
99              천호역 데이앤데이
Name: 검색어, Length: 100, dtype: object

In [49]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
user_review_id = {}
review_json = {}
image_json = {}
item_meta = {}
count=0
for item in items:
    
    driver.get("https://www.google.com/maps/") # 구글 지도 접속하기
    searchbox = driver.find_element_by_css_selector("input#searchboxinput") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_class_name("searchbox-searchbutton") # 검색버튼 누르기
    searchbutton.click()
    time.sleep(5)
  
    
    
    if len(driver.find_elements_by_xpath("//button[@jsaction='pane.rating.moreReviews']")) != 0:
        print("식당 1개 존재")
        
        reviewbutton = driver.find_element_by_xpath("//button[@jsaction='pane.rating.moreReviews']")
        res_name = driver.find_element_by_xpath("//div[@class='section-layout section-layout-root']").get_attribute("aria-label")
        print("{} vs {}".format(res_name,item))
        reviewbutton.click()
    elif len(driver.find_elements_by_xpath("//a[@class='place-result-container-place-link']")) > 0:
        if driver.find_elements_by_xpath("//a[@class='place-result-container-place-link']")[0].get_attribute("aria-label") == item.split()[0]:
            print("{} 검색".format(item.split()[0]))
            continue
        elif len(driver.find_elements_by_xpath("//button[@jsaction='pane.rating.moreReviews']")) != 0:
            print("여러 식당 검색")
            firstbox = driver.find_elements_by_xpath("//a[@class='place-result-container-place-link']")[0]
            res_name = firstbox.get_attribute("aria-label")
            print("{} vs {}".format(firstbox.get_attribute("aria-label"),item))
            driver.get(firstbox.get_attribute("href"))
            time.sleep(5)
            reviewbutton = driver.find_element_by_xpath("//button[@jsaction='pane.rating.moreReviews']")
            reviewbutton.click()
        else:
            print("리뷰 존재 X")
            count+=1
            continue
    else:
        print("식당 존재 X")
        count+=1
        continue

    time.sleep(2)
    # 리뷰 전체 뽑기 위해 스크롤 다운
    
    for num in range(30):
        try:
            driver.find_element_by_class_name('section-loading').click()
            time.sleep(2)
        except:
            print(item + " 스크롤 다운 완료")
            time.sleep(2)
            break

    spread_review = driver.find_elements_by_xpath("//button[@jsaction='pane.review.expandReview']") 
    time.sleep(2)
    for i in range(len(spread_review)): 
        isTrue = spread_review[i].is_displayed() # 보이는 것인지를 확인 
        print("Element is visible? " + str(isTrue)) 
        if isTrue: 
            spread_review[i].click() 
            print(str(i)+"th more button is clicked and wait 2 secs...") 
            time.sleep(2)
        else:
            print("Element is visible? " + str(isTrue)) 


    # 시간 지연
    time.sleep(2)
    user = driver.find_elements_by_xpath("//a[@class='section-review-reviewer-link']")
    user_review = driver.find_elements_by_xpath("//button[@class='section-review-action-menu']")
    date = driver.find_elements_by_xpath("//span[@class='section-review-publish-date']")
    rating = driver.find_elements_by_xpath("//span[@class='section-review-stars']")

    time.sleep(2)
    review = driver.find_elements_by_xpath("//span[@class='section-review-text']")


    # Image

    image = driver.find_elements_by_xpath("//button[@class='section-review-photo']")
    review_json[item] = {}
    user_review_id[item] = {}
    image_json[item] = {}
    item_meta[item] = {"Name" : res_name}
 
    for i in range(len(date)):
        if review[i].text is not "":
            user_elem = re.findall("\d+",user[i].get_attribute("href"))[0]
            item_elem = item
            rating_elem = re.findall("\d+",rating[i].get_attribute('aria-label'))[0]
            date_elem = date[i].text
            row = {"UserID":user_elem,"ItemID":item_elem,"Rating":rating_elem,"Timestamp":date_elem}
            row = pd.DataFrame(row, index=[i])
            rating_df = rating_df.append(row,ignore_index=True)
            user_review_id[item][user_elem] = user_review[i].get_attribute("data-review-id")
            review_json[item][user_review[i].get_attribute("data-review-id")] = review[i].text
    for i in range(len(image)):
        image_json[item][image[i].get_attribute("data-review-id")] = re.findall(r'\"(.+?)\"',image[i].get_attribute('style'))[0]

# 크롬창 닫기
driver.close()

식당 존재 X
홍대입구역 검색
홍대입구역 검색
리뷰 존재 X
식당 1개 존재
Bisit /비지트 vs 방배역 비지트
방배역 비지트 스크롤 다운 완료
식당 존재 X
리뷰 존재 X
리뷰 존재 X
식당 1개 존재
다향할매집 vs 서초역 다향
서초역 다향 스크롤 다운 완료
식당 존재 X
식당 1개 존재
든든한우家 vs 남부터미널역 든든한우家
남부터미널역 든든한우家 스크롤 다운 완료
리뷰 존재 X
식당 1개 존재
옥돌영양탕 vs 아차산역 옥돌영양탕
아차산역 옥돌영양탕 스크롤 다운 완료
식당 1개 존재
양푼이 김치찌개 vs 몽촌토성역 양푼이김치찌개
몽촌토성역 양푼이김치찌개 스크롤 다운 완료
식당 존재 X
식당 존재 X
리뷰 존재 X
식당 존재 X
식당 1개 존재
제일곱창 vs 왕십리역 왕십리제일곱창
Element is visible? True
0th more button is clicked and wait 2 secs...
Element is visible? True
1th more button is clicked and wait 2 secs...
Element is visible? True
2th more button is clicked and wait 2 secs...
Element is visible? True
3th more button is clicked and wait 2 secs...
Element is visible? True
4th more button is clicked and wait 2 secs...
Element is visible? True
5th more button is clicked and wait 2 secs...
리뷰 존재 X
식당 1개 존재
CGV 영등포 vs 영등포역 씨지브이영등포골드클래스
Element is visible? True
0th more button is clicked and wait 2 secs...
Element is visible? True
1th more button is clicked and wait 2 secs

In [50]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,109317799373601543644,서초역 다향,2,1년 전
1,105402685935557923062,서초역 다향,5,2년 전
2,115410654106362321091,남부터미널역 든든한우家,4,1년 전
3,104186100072005724465,남부터미널역 든든한우家,4,1년 전
4,118091836624399905856,남부터미널역 든든한우家,5,1년 전
...,...,...,...,...
1015,105293097763616384297,상왕십리역 Mr박의돈워리,5,3년 전
1016,117023752037751136416,상왕십리역 Mr박의돈워리,5,2년 전
1017,101786844191814487795,상왕십리역 Mr박의돈워리,5,2년 전
1018,110263048983706653386,상왕십리역 Mr박의돈워리,5,2년 전


In [51]:
user_review_id

{'방배역 비지트': {},
 '서초역 다향': {'109317799373601543644': 'ChdDSUhNMG9nS0VJQ0FnSURVbUxLb293RRAB',
  '105402685935557923062': 'ChdDSUhNMG9nS0VJQ0FnSUNRcHZ2MXpBRRAB'},
 '남부터미널역 든든한우家': {'115410654106362321091': 'ChZDSUhNMG9nS0VJQ0FnSUMwbHVYdUdnEAE',
  '104186100072005724465': 'ChZDSUhNMG9nS0VJQ0FnSUQ0LVlYWFdBEAE',
  '118091836624399905856': 'ChZDSUhNMG9nS0VJQ0FnSUNZejZUYkh3EAE',
  '106976535237596500734': 'ChZDSUhNMG9nS0VJQ0FnSURVbTVyQVlBEAE',
  '107020742025917540795': 'ChdDSUhNMG9nS0VJQ0FnSURFZzl5T2pBRRAB',
  '103254528320673468300': 'ChZDSUhNMG9nS0VJQ0FnSURrN3BPTkdBEAE',
  '106655447816758179248': 'ChdDSUhNMG9nS0VJQ0FnSUNBN3IzQ2xnRRAB',
  '102745841966467493043': 'ChdDSUhNMG9nS0VJQ0FnSUNzajZ1bzZ3RRAB',
  '112389665171853372684': 'ChdDSUhNMG9nS0VJQ0FnSURVaktHLXdBRRAB',
  '101122495290686958153': 'ChdDSUhNMG9nS0VJQ0FnSURRaTVMWm1nRRAB',
  '117897698324874401378': 'ChdDSUhNMG9nS0VJQ0FnSUNvMktpMHBRRRAB',
  '117710208919776617293': 'ChZDSUhNMG9nS0VJQ0FnSUMwOE1LWVhBEAE',
  '108053585642187867218'

In [52]:
review_json

{'방배역 비지트': {},
 '서초역 다향': {'ChdDSUhNMG9nS0VJQ0FnSURVbUxLb293RRAB': '내입맛엔 별로',
  'ChdDSUhNMG9nS0VJQ0FnSUNRcHZ2MXpBRRAB': '항상 맛있는곳'},
 '남부터미널역 든든한우家': {'ChZDSUhNMG9nS0VJQ0FnSUMwbHVYdUdnEAE': '맛있다. 친절하다.\n홀 직원수가 적어서 그런지 세네번 불러야 주문 가능.\n발렛비 2천원.',
  'ChZDSUhNMG9nS0VJQ0FnSUQ0LVlYWFdBEAE': '룸이 있어 회식자리로 괜찮습니다. 단 가격대가 좀 있는편 ㅎㅎㅎ',
  'ChZDSUhNMG9nS0VJQ0FnSUNZejZUYkh3EAE': '육개장 점심시간에 먹으러 가는데 깔끔한 밑반찬 5가지에 육개장 정갈하고 맛난건 기본.\n기름기는 좀 있지만 8천원에 고급진 육개장이~~~',
  'ChZDSUhNMG9nS0VJQ0FnSURVbTVyQVlBEAE': '조용하고 방들이 입식으로 구획이 잘 되어있음. 주차장도 넓음.',
  'ChdDSUhNMG9nS0VJQ0FnSURFZzl5T2pBRRAB': '가격대비 서비스나 음식이 보통입니다',
  'ChZDSUhNMG9nS0VJQ0FnSURrN3BPTkdBEAE': '가격대비 고기가 너무 적음',
  'ChdDSUhNMG9nS0VJQ0FnSUNBN3IzQ2xnRRAB': '남편이랑 오랜만에 둘이 외식했는데, 둘다 만족하고 왔습니다. 깨끗하고, 맛있고, 대충봤는데 실내골프장도 위에 있더라구요? 담엔 남편이랑 실내골프장이용하고 은근히 고기먹으러 데려가려구요.',
  'ChdDSUhNMG9nS0VJQ0FnSUNzajZ1bzZ3RRAB': '맛있어요~^^',
  'ChdDSUhNMG9nS0VJQ0FnSURVaktHLXdBRRAB': '육회비빔밥 맛있어요',
  'ChdDSUhNMG9nS0VJQ0FnSURRaTVMWm1nRRAB': '모임하기 좋습니다',
  'ChdDSUhNMG9nS0VJQ0FnSUNvMktpMHBRRRA

In [53]:
image_json

{'방배역 비지트': {},
 '서초역 다향': {},
 '남부터미널역 든든한우家': {'ChZDSUhNMG9nS0VJQ0FnSUQ0LVlYWFdBEAE': 'https://lh5.googleusercontent.com/p/AF1QipOmgoCQ7W3JSalYlvsW4yZHbcN4pUSjQ6AfXQrA=w750-h750-p-k-no'},
 '아차산역 옥돌영양탕': {'ChdDSUhNMG9nS0VJQ0FnSUR3eGFTTHJBRRAB': 'https://lh5.googleusercontent.com/p/AF1QipNzU3oYf-YQDjb9-tIA3NXoRVS-SxwDbNpdN7Ez=w750-h750-p-k-no'},
 '몽촌토성역 양푼이김치찌개': {'ChZDSUhNMG9nS0VJQ0FnSURNbGVqNFlnEAE': 'https://lh5.googleusercontent.com/p/AF1QipP6ea5YlxRGezepWNh9uOLV14yWnLzF1kmd0Inx=w750-h750-p-k-no'},
 '왕십리역 왕십리제일곱창': {'ChdDSUhNMG9nS0VJQ0FnSUNLa0s2UHZRRRAB': 'https://lh5.googleusercontent.com/p/AF1QipMJ4wcR7WzPy4mk-x_MVELj4XFxlOvo_MW4Vj2S=w750-h750-p-k-no',
  'ChdDSUhNMG9nS0VJQ0FnSURTM3NQT2dRRRAB': 'https://lh5.googleusercontent.com/p/AF1QipOTnRQWRBns7TZGyEhlAPc0b3J3uQ5VVOIhCqpZ=w750-h750-p-k-no',
  'ChdDSUhNMG9nS0VJQ0FnSUR5OU1DaG5RRRAB': 'https://lh5.googleusercontent.com/p/AF1QipMUIKT8gO-nTbUqQ9_F4epFG2_CC-8yQ4KDV6gK=w750-h750-p-k-no',
  'ChZDSUhNMG9nS0VJQ0FnSUN5MXNtU093EAE': 'https

In [54]:
item_meta

{'방배역 비지트': {'Name': 'Bisit /비지트'},
 '서초역 다향': {'Name': '다향할매집'},
 '남부터미널역 든든한우家': {'Name': '든든한우家'},
 '아차산역 옥돌영양탕': {'Name': '옥돌영양탕'},
 '몽촌토성역 양푼이김치찌개': {'Name': '양푼이 김치찌개'},
 '왕십리역 왕십리제일곱창': {'Name': '제일곱창'},
 '영등포역 씨지브이영등포골드클래스': {'Name': 'CGV 영등포'},
 '신림역 수제비,잔치': {'Name': '수제비잔치'},
 '광운대역 놀부부대찌개철판구이성북역점': {'Name': '놀부부대찌개 성북역점'},
 '방화역 울엄니밥상': {'Name': '울엄니밥상'},
 '가양역 열라닭발': {'Name': '열라닭발&닭갈비'},
 '청량리역 본죽': {'Name': '본죽 청량리점'},
 '삼양사거리역 봉구비어삼양동사거리점': {'Name': '봉구비어삼양동사거리점'},
 '삼성역 대가집': {'Name': '대가집'},
 '가좌역 손가네빈대떡': {'Name': '손가네빈대떡'},
 '사평역 분노지': {'Name': '분노지'},
 '가락시장역 인생극장쪽갈비': {'Name': '인생극장 가락점'},
 '경복궁역 대송': {'Name': '대송'},
 '신이문역 등대횟집': {'Name': '등대횟집'},
 '청량리역 던킨도너츠동대문홈플러스': {'Name': '던킨도너츠 청량리역사점'},
 '삼성역 하레': {'Name': '하레이자카야'},
 '잠실역 부탄츄': {'Name': '부탄츄 롯데월드몰점'},
 '까치산역 우성커피숍': {'Name': '우성커피숍'},
 '노량진역 대추나무복아구': {'Name': '대추나무복집'},
 '상왕십리역 산수골추어감자탕': {'Name': '산수골 추어탕 감자탕'},
 '상왕십리역 Mr박의돈워리': {'Name': 'Mr박의돈워리'},
 '방화역 버블베어': {'Name': '버블베어'}}

In [55]:
count/len(items)

0.2